# ReadME

1) Make sure you downloaded the raw data and ran the EDA notebook
2) Now download the neo4j community edition
3) Tried using the python api but the upload times were horrendous. So you will need to upload via CLI, the command is there in 

In [5]:
import os
import sqlite3
import pandas as pd
import math
import concurrent

orders_df = pd.read_csv('../data/raw_data/orders.csv')
order_products_prior = pd.read_csv('../data/raw_data/order_products__prior.csv')
order_products_train = pd.read_csv('../data/raw_data/order_products__train.csv')
df_products = pd.read_csv('../data/raw_data/products.csv')
df_aisles = pd.read_csv('../data/raw_data/aisles.csv')
df_departments = pd.read_csv('../data/raw_data/departments.csv')

# Database path
db_path = '../data/consumption_data/INSTACART.db'


if not os.path.exists(db_path):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    cur.execute('''
        CREATE TABLE aisles (
            aisle_id INTEGER PRIMARY KEY,
            aisle TEXT
        );
    ''')
    cur.execute('''
        CREATE TABLE departments (
            department_id INTEGER PRIMARY KEY,
            department TEXT
        );
    ''')
    cur.execute('''
        CREATE TABLE products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT,
            aisle_id INTEGER,
            department_id INTEGER,
            FOREIGN KEY (aisle_id) REFERENCES aisles(aisle_id),
            FOREIGN KEY (department_id) REFERENCES departments(department_id)
        );
    ''')
    cur.execute('''
        CREATE TABLE orders (
            order_id INTEGER PRIMARY KEY,
            user_id INTEGER,
            eval_set TEXT,
            order_number INTEGER,
            order_dow INTEGER,
            order_hour_of_day INTEGER,
            days_since_prior_order REAL
        );
    ''')
    cur.execute('''
        CREATE TABLE order_products (
            order_id INTEGER,
            product_id INTEGER,
            add_to_cart_order INTEGER,
            reordered INTEGER,
            FOREIGN KEY (order_id) REFERENCES orders(order_id),
            FOREIGN KEY (product_id) REFERENCES products(product_id)
        );
    ''')

    conn.commit()

    orders_df.to_sql(name='orders', con=conn, if_exists='replace', index=False)
    order_products_prior.to_sql(name='order_products', con=conn, if_exists='replace', index=False)
    order_products_train.to_sql(name='order_products', con=conn, if_exists='append', index=False)
    df_products.to_sql(name='products', con=conn, if_exists='replace', index=False)
    df_aisles.to_sql(name='aisles', con=conn, if_exists='replace', index=False)
    df_departments.to_sql(name='departments', con=conn, if_exists='replace', index=False)
    
    conn.close()



![](graph.svg)